<a href="https://colab.research.google.com/github/CamillaMilli/ADM-HMW1/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
#movielens_20m_dataset_path = kagglehub.dataset_download('grouplens/movielens-20m-dataset')
print('Data source import complete.')


Data source import complete.


In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/genome_scores.csv
/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/link.csv
/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/rating.csv
/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/movie.csv
/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/tag.csv
/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/genome_tags.csv


In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


https://github.com/rapidsai/cudf?tab=readme-ov-file#pip

In [8]:
pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [10]:
import rmm
import cudf
rmm.reinitialize(pool_allocator=True, managed_memory=True)
print("RAPIDS cuDF is using GPU memory:", rmm.is_initialized())

RAPIDS cuDF is using GPU memory: True


In [11]:
import cupy as cp

def check_memory():
    memory_pool = cp.get_default_memory_pool()
    total_memory = cp.cuda.Device(0).mem_info[0]  # Total memory available
    used_memory = memory_pool.used_bytes()       # Memory allocated by the pool
    free_memory = total_memory - used_memory
    print(f"GPU Memory - Total: {total_memory / (1024**3):.2f} GB, Used: {used_memory / (1024**3):.2f} GB, Free: {free_memory / (1024**3):.2f} GB")

check_memory()

GPU Memory - Total: 14.65 GB, Used: 0.00 GB, Free: 14.65 GB


In [12]:
movielens_20m_dataset_path = kagglehub.dataset_download('grouplens/movielens-20m-dataset')
#Remove the extra space at the start of the loop
for dirname, _, filenames in os.walk('/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1'):
    #Your Code Logic for inside the for loop
    pass # Placeholder to show loop block

In [15]:
ratings_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/rating.csv")
links_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/link.csv")
genome_tags_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/genome_tags.csv")
genome_scores_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/genome_scores.csv")
tags_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/tag.csv")
movies_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/movie.csv")


In [13]:
check_memory()

GPU Memory - Total: 14.65 GB, Used: 0.00 GB, Free: 14.65 GB


In [16]:
print("ratings_df:",ratings_df.columns)
print("links_df:",links_df.columns)
print("genome_tags_df:",genome_tags_df.columns)
print("genome_scores_df:",genome_scores_df.columns)
print("tags_df:",tags_df.columns)
print("movies_df:",movies_df.columns)

ratings_df: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
links_df: Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')
genome_tags_df: Index(['tagId', 'tag'], dtype='object')
genome_scores_df: Index(['movieId', 'tagId', 'relevance'], dtype='object')
tags_df: Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')
movies_df: Index(['movieId', 'title', 'genres'], dtype='object')


In [17]:
import gc
rmm.reinitialize(pool_allocator=True, managed_memory=True)  # Reset GPU memory
movies_links = movies_df.merge(links_df, on="movieId", how="left")
del links_df  # Free up memory
gc.collect()
movies_links.head()

,movieId,title,genres,imdbId,tmdbId
0,2293,Shadrach (1998),Drama,144604,73351
1,2294,Antz (1998),Adventure|Animation|Children|Comedy|Fantasy,120587,8916
2,2295,"Impostors, The (1998)",Comedy,120823,28134
3,2296,"Night at the Roxbury, A (1998)",Comedy,120770,9429
4,2297,What Dreams May Come (1998),Adventure|Drama|Fantasy|Romance,120889,12159


In [18]:
# Merge ratings with movies
rmm.reinitialize(pool_allocator=True, managed_memory=True)  # Reset GPU memory
ratings_movies = ratings_df.merge(movies_links, on="movieId", how="left")
del ratings_df, movies_links  # Free up memory
gc.collect()
ratings_movies.head()

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId
0,11,37380,2.0,2009-01-01 05:26:37,Doom (2005),Action|Horror|Sci-Fi,419706,8814
1,11,37386,1.5,2009-01-01 04:27:36,Aeon Flux (2005),Action|Sci-Fi,402022,8202
2,11,37830,5.0,2009-01-01 05:40:38,Final Fantasy VII: Advent Children (2004),Action|Adventure|Animation|Fantasy|Sci-Fi,385700,647
3,11,39446,1.5,2009-01-02 00:37:45,Saw II (2005),Horror|Thriller,432348,215
4,11,40339,3.0,2009-01-01 05:51:58,Chicken Little (2005),Action|Adventure|Animation|Children|Comedy|Sci-Fi,371606,9982


In [19]:
rmm.reinitialize(pool_allocator=True, managed_memory=True)  # Reset GPU memory
# Merge genome scores with genome tags
genome_data = genome_scores_df.merge(genome_tags_df, on="tagId", how="left")
del genome_tags_df  # Free up memory
gc.collect()
genome_data.head()

,movieId,tagId,relevance,tag
0,2,537,0.04125,immigrants
1,2,538,0.08400,immortality
2,2,539,0.01425,incest
3,2,540,0.05825,independent film
4,2,541,0.00700,india


In [20]:
rmm.reinitialize(pool_allocator=True, managed_memory=True)
# Merge tags with movies
tags_movies = tags_df.merge(movies_df, on="movieId", how="left")
del tags_df  # Free up memory
gc.collect()
tags_movies.head()

,userId,movieId,tag,timestamp,title,genres
0,1741,924,adapted from:book,2007-04-28 23:15:36,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi
1,1741,924,artificial intelligence,2007-06-23 16:31:36,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi
2,1741,924,author:Arthur C. Clarke,2007-06-23 16:31:31,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi
3,1741,924,related:2001,2009-08-09 02:44:31,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi
4,1741,924,setting:space/space ship,2008-10-11 02:00:42,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi


In [21]:
check_memory()

GPU Memory - Total: 9.14 GB, Used: 0.00 GB, Free: 9.14 GB


In [22]:
print("ratings_movies:",ratings_movies.columns)
print("genome_data:",genome_data.columns)
print("tags_movies:",tags_movies.columns)

ratings_movies: Index(['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres', 'imdbId',
       'tmdbId'],
      dtype='object')
genome_data: Index(['movieId', 'tagId', 'relevance', 'tag'], dtype='object')
tags_movies: Index(['userId', 'movieId', 'tag', 'timestamp', 'title', 'genres'], dtype='object')


In [23]:
ratings_movies = ratings_movies[["userId", "movieId", "rating", "title", "genres","timestamp"]]
genome_data = genome_data[["movieId", "tag", "relevance"]]
tags_movies = tags_movies[["userId", "movieId", "tag"]]
#Remove repeated columns

In [24]:
print("ratings_movies:",ratings_movies.info())
print("genome_data:",genome_data.info())
print("tags_movies:",tags_movies.info())

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype
---  ------     -----
 0   userId     int64
 1   movieId    int64
 2   rating     float64
 3   title      object
 4   genres     object
 5   timestamp  object
dtypes: float64(1), int64(2), object(3)
memory usage: 1.8+ GB
ratings_movies: None
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype
---  ------     -----
 0   movieId    int64
 1   tag        object
 2   relevance  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 335.2+ MB
genome_data: None
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 465426 entries, 0 to 465425
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userId   465426 non-null  int64
 1   movieId  465426 non-null  int64
 2   tag      465426 non-null  object
dtypes: int

In [25]:
ratings_movies.isnull().sum()

userId       0
movieId      0
rating       0
title        0
genres       0
timestamp    0
dtype: int64

In [26]:
ratings_movies.drop_duplicates()

,userId,movieId,rating,title,genres,timestamp
0,11,37380,2.0,Doom (2005),Action|Horror|Sci-Fi,2009-01-01 05:26:37
1,11,37386,1.5,Aeon Flux (2005),Action|Sci-Fi,2009-01-01 04:27:36
2,11,37830,5.0,Final Fantasy VII: Advent Children (2004),Action|Adventure|Animation|Fantasy|Sci-Fi,2009-01-01 05:40:38
3,11,39446,1.5,Saw II (2005),Horror|Thriller,2009-01-02 00:37:45
4,11,40339,3.0,Chicken Little (2005),Action|Adventure|Animation|Children|Comedy|Sci-Fi,2009-01-01 05:51:58
...,...,...,...,...,...,...
20000258,138493,2696,4.0,"Dinner Game, The (Dîner de cons, Le) (1998)",Comedy,2009-11-13 17:55:27
20000259,138493,2710,3.0,"Blair Witch Project, The (1999)",Drama|Horror|Thriller,2009-10-17 18:56:32
20000260,138493,2716,4.0,Ghostbusters (a.k.a. Ghost Busters) (1984),Action|Comedy|Sci-Fi,2009-11-16 16:52:05
20000261,138493,2422,1.0,"Karate Kid, Part III, The (1989)",Action|Adventure|Children|Drama,2009-11-13 17:53:07


this merged dataset in clean , there aren't missing values

In [25]:
tags_movies.isnull().sum()

userId     0
movieId    0
tag        0
dtype: int64

In [26]:
genome_data.isnull().sum()

movieId      0
tag          0
relevance    0
dtype: int64

In [54]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [59]:
print(ratings_movies.shape)  # Verifica il numero di righe e colonne
print(ratings_movies.head())  # Verifica le prime righe per confermare che il merge è riuscito


(20000263, 8)
   userId  movieId  rating            timestamp  \
0      74     1094     5.0  2001-01-09 22:52:35   
1      74     1104     4.0  2001-01-09 22:52:35   
2      74     1131     4.0  2001-01-09 22:59:15   
3      74     1172     5.0  2001-01-09 22:55:58   
4      74     1179     4.0  2001-01-09 23:02:11   

                                            title                  genres  \
0                         Crying Game, The (1992)  Drama|Romance|Thriller   
1                Streetcar Named Desire, A (1951)                   Drama   
2                         Jean de Florette (1986)           Drama|Mystery   
3  Cinema Paradiso (Nuovo cinema Paradiso) (1989)                   Drama   
4                            Grifters, The (1990)   Crime|Drama|Film-Noir   

   imdbId  tmdbId  
0  104036   11386  
1   44081     702  
2   91288    4480  
3   95765   11216  
4   99703   18129  


In [60]:
ratings_movies = ratings_movies.dropna(subset=['movieId'])


In [61]:
grouped = ratings_movies.groupby('userId').size()
print(grouped)


userId
92922     156
75026     246
12513      42
52602      27
23593      41
         ... 
131903     22
111596     30
55593      42
109079     53
42942     161
Length: 138493, dtype: int64


In [27]:
import cudf

# Convert the cuDF DataFrame to a Pandas DataFrame
ratings_movies_pd = ratings_movies.to_pandas()

# Perform the groupby and apply operations using Pandas
user_movie_groups = ratings_movies_pd.groupby('userId')['movieId'].apply(list).reset_index()

# If you need to convert it back to a cuDF DataFrame:
user_movie_groups = cudf.from_pandas(user_movie_groups)

In [28]:
import cudf
import numpy as np

all_movies = ratings_movies['movieId'].unique().to_pandas() # Convert to Pandas Series

# Crea una matrice utente-film con 1 se l'utente ha visto il film, altrimenti 0
user_movie_matrix = np.zeros((len(user_movie_groups), len(all_movies)))

# Mappa i film unici a indici
movie_index = {movie: idx for idx, movie in enumerate(all_movies)} # Now iterable

# Convert the cuDF DataFrame to a Pandas DataFrame for iteration
user_movie_groups_pd = user_movie_groups.to_pandas()

# Popola la matrice binaria using the Pandas DataFrame
for idx, row in user_movie_groups_pd.iterrows():
    user_id = row['userId']
    movie_ids = row['movieId']
    for movie in movie_ids:
        user_movie_matrix[idx, movie_index[movie]] = 1

# La matrice user_movie_matrix ora è una matrice binaria


In [ ]:
# Parametri per MinHash
num_hash_functions = 50  # Numero di funzioni hash da generare
num_users, num_movies = user_movie_matrix.shape

# Inizializza la matrice delle firme MinHash
signature_matrix = np.full((num_hash_functions, num_users), np.inf)

# Funzione per generare hash function lineari: h(x) = (ax + b) % num_movies
def generate_hash_functions(num_funcs, max_val):
    hash_funcs = []
    np.random.seed(42)  # Per riproducibilità
    for _ in range(num_funcs):
        a = np.random.randint(1, max_val)
        b = np.random.randint(0, max_val)
        hash_funcs.append((a, b))
    return hash_funcs

# Generazione delle hash function
hash_funcs = generate_hash_functions(num_hash_functions, num_movies)

# Applicazione di MinHash per ogni film (colonna)
for movie_idx in range(num_movies):
    if user_movie_matrix[:, movie_idx].sum() == 0:
        continue  # Salta se nessun utente ha visto questo film

    for func_idx, (a, b) in enumerate(hash_funcs):
        # Per ogni film, calcola il valore hash per ogni utente
        hash_value = (a * (movie_idx + 1) + b) % num_movies

        for user_idx in range(num_users):
            if user_movie_matrix[user_idx, movie_idx] == 1:
                # Aggiorna la firma MinHash per l'utente
                signature_matrix[func_idx, user_idx] = min(signature_matrix[func_idx, user_idx], hash_value)

# Converti la matrice delle firme in un DataFrame
signature_df = pd.DataFrame(signature_matrix, columns=user_movie_groups['userId'])

print(signature_df.head())


In [ ]:
# Funzione per calcolare la similarità Jaccard tra due firme MinHash
def jaccard_similarity(sig1, sig2):
    return np.sum(sig1 == sig2) / len(sig1)

# Esempio di calcolo della similarità tra due utenti (sostituisci con gli userId reali)
user1 = 1
user2 = 2
similarity = jaccard_similarity(signature_df[user1], signature_df[user2])
print(f"Similarità Jaccard tra User {user1} e User {user2}: {similarity:.2f}")
